In [35]:
import pandas as pd

In [36]:
meteorite_data = pd.read_csv('meteorites.csv')

In [37]:
meteorite_data.head()

,name,id,name_type,class,mass,fall,year,lat,long,geolocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)"


In [38]:
meteorite_data.isnull().sum()

name              0
id                0
name_type         0
class             0
mass            131
fall              0
year            291
lat            7315
long           7315
geolocation    7315
dtype: int64

In [39]:
# Drop rows with missing values
print(meteorite_data.shape)
meteorite_data = meteorite_data.dropna()
print(meteorite_data.shape)

(45716, 10)
(38115, 10)


In [40]:
# Incorrect data points according to kaggle -> https://www.kaggle.com/datasets/nasa/meteorite-landings
meteorite_data = meteorite_data[(meteorite_data['year'] >= 860) & (meteorite_data['year'] <= 2016)]
meteorite_data = meteorite_data[(meteorite_data['long'] <= 180) & (meteorite_data['long'] >= -180)]
meteorite_data = meteorite_data[(meteorite_data['lat'] != 0) & (meteorite_data['lat'] != 0)]
meteorite_data.shape

(31705, 10)

In [41]:
meteorite_data.to_csv('meteorites_clean.csv', index=False)

In [42]:
import reverse_geocoder as rg

def get_country_from_coords(lat, lon):
    results = rg.search((lat, lon))
    country_name = results[0]["cc"]
    return country_name

meteorite_data['countrycode'] = meteorite_data.apply(lambda row: get_country_from_coords(row['lat'], row['long']), axis=1)

In [43]:
meteorite_data.head()

,name,id,name_type,class,mass,fall,year,lat,long,geolocation,countrycode
5468,Chupaderos,5363,Valid,"Iron, IIIAB",24300000.00,Found,1852.0,27.00000,-105.10000,"(27.0, -105.1)",MX
1754,Allan Hills 83042,562,Valid,H3.6,0.50,Found,1983.0,-76.81765,158.19001,"(-76.81765, 158.19001)",AQ
35751,Ramlat as Sahmah 340,55432,Valid,L6,414.10,Found,2010.0,20.79315,55.46493,"(20.79315, 55.46493)",OM
5128,Black Rock 001,32765,Valid,L6,152.00,Found,2003.0,40.86717,-119.18352,"(40.86717, -119.18352)",US
40531,Yamato 791308,26657,Valid,H4/5,2.84,Found,1979.0,-71.50000,35.66667,"(-71.5, 35.66667)",ZA


In [44]:
continent_mapping = pd.read_csv('continentmapping.csv')

In [45]:
continent_mapping.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [52]:
merged_data = pd.merge(meteorite_data, continent_mapping, how='left', left_on='countrycode', right_on='alpha-2')

# Rename the columns to more descriptive names
merged_data.rename(columns={'name_x': 'name','name_y': 'countryname', 'region': 'continent'}, inplace=True)

# Drop the 'alpha-2', 'alpha-3', 'country-code', 'iso_3166-2', 'region-code',
# 'sub-region-code', and 'intermediate-region-code' columns, since we don't need them
merged_data.drop(['alpha-2', 'alpha-3', 'country-code', 'iso_3166-2',
                  'region-code', 'sub-region-code', 'intermediate-region-code'],
                 axis=1, inplace=True)
merged_data.loc[merged_data['countryname'] == 'Antarctica', 'continent'] = 'Antarctica'

In [53]:
merged_data.to_csv('meteorites_clean.csv')

,name,id,name_type,class,mass,fall,year,lat,long,geolocation,countrycode,countryname,continent,sub-region,intermediate-region
0,Chupaderos,5363,Valid,"Iron, IIIAB",24300000.00,Found,1852.0,27.00000,-105.10000,"(27.0, -105.1)",MX,Mexico,Americas,Latin America and the Caribbean,Central America
1,Allan Hills 83042,562,Valid,H3.6,0.50,Found,1983.0,-76.81765,158.19001,"(-76.81765, 158.19001)",AQ,Antarctica,Antarctica,NaN,NaN
2,Ramlat as Sahmah 340,55432,Valid,L6,414.10,Found,2010.0,20.79315,55.46493,"(20.79315, 55.46493)",OM,Oman,Asia,Western Asia,NaN
3,Black Rock 001,32765,Valid,L6,152.00,Found,2003.0,40.86717,-119.18352,"(40.86717, -119.18352)",US,United States,Americas,Northern America,NaN
4,Yamato 791308,26657,Valid,H4/5,2.84,Found,1979.0,-71.50000,35.66667,"(-71.5, 35.66667)",ZA,South Africa,Africa,Sub-Saharan Africa,Southern Africa


In [ ]:
# Change the year from float to int so that vega lite can recognise it
meteorite_data_clean = pd.read_csv('meteorites_clean.csv')
meteorite_data_clean['year'] = pd.to_numeric(meteorite_data_clean['year'], downcast='unsigned', errors='coerce')
meteorite_data_clean.to_csv('meteorites_clean_2.csv', index=False)